# Hyperparameters optimisation using bayesian search

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from bayes_opt import BayesianOptimization

In [2]:

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

11490434/11490434 [==============================] - 1s 0us/step


## Define the model architecture

In [8]:



def create_model(learning_rate, dropout_rate, num_filters, kernel_size):
    model = Sequential()
    model.add(Conv2D(num_filters, kernel_size=kernel_size, activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


## Define the hyperparameter search space

In [4]:

pbounds = {'learning_rate': (0.0001, 0.01), 'dropout_rate': (0.0, 0.5), 'num_filters': (16, 64), 'kernel_size': (3, 5)}

## Define the objective function to maximize

In [5]:

def objective(learning_rate, dropout_rate, num_filters, kernel_size):
    model = create_model(learning_rate, dropout_rate, int(num_filters), int(kernel_size))

    history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=128, verbose=0)

    return history.history['val_accuracy'][-1]

## Create a Bayesian optimization object and run the search

In [6]:

optimizer = BayesianOptimization(f=objective, pbounds=pbounds, random_state=42)
optimizer.maximize(n_iter=10)

|   iter    |  target   | dropou... | kernel... | learni... | num_fi... |
-------------------------------------------------------------------------


| 1         | 0.9881    | 0.1873    | 4.901     | 0.007347  | 44.74     |


| 2         | 0.9853    | 0.07801   | 3.312     | 0.000675  | 57.58     |


| 3         | 0.9884    | 0.3006    | 4.416     | 0.0003038 | 62.56     |


| 4         | 0.9851    | 0.4162    | 3.425     | 0.0019    | 24.8      |
| 5         | 0.987     | 0.1521    | 4.05      | 0.004376  | 29.98     |


...

| 9         | 0.9878    | 0.2694    | 4.48      | 0.0062    | 62.7      |


| 10        | 0.9874    | 0.09665   | 4.805     | 0.009593  | 44.87     |


| 11        | 0.9903    | 0.4692    | 4.413     | 0.004556  | 63.93     |


| 12        | 0.9893    | 0.3876    | 4.764     | 0.002349  | 33.88     |
| 13        | 0.9883    | 0.4676    | 4.35      | 0.003089  | 63.8      |


| 14        | 0.9869    | 0.003152  | 3.919     | 0.009769  | 41.28     |
| 15        | 0.9859    | 0.3713    | 4.05      | 0.003073  | 16.94     |


In [7]:
# Print the best set of hyperparameters found
print(optimizer.max)


{'target': 0.9902999997138977, 'params': {'dropout_rate': 0.4691568708481438, 'kernel_size': 4.412652249431585, 'learning_rate': 0.0045563272624315814, 'num_filters': 63.92646483894076}}
